# Day 4: Giant Squid ---

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

`7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1`

```
22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```

After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```

After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```

Finally, 24 is drawn:

```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

In [2]:
input_data = '../data/day_04_puzzle_1.csv'

In [3]:
with open(input_data, 'r') as f:
    bingo = f.readlines()

In [11]:
marked = bingo[0].replace('\n','').split(',')
marked[:10]

['17', '58', '52', '49', '72', '33', '55', '73', '27', '69']

In [15]:
bingo[2:10]

['59 98 84 27 56\n',
 '17 35 18 64 34\n',
 '62 16 74 26 55\n',
 '21 99  1 19 93\n',
 '65 68 53 24 73\n',
 '\n',
 ' 1 86 98 16  6\n',
 '93 69 33 49 71\n']

In [268]:
import numpy as np
from typing import Union

class BingoBoard:
    def __init__(self, board: Union[np.array, list]) -> None:
        if isinstance(board, list):
            board = np.array(board)
        elif isinstance(board, np.array):
            pass
        else:
            raise TypeError("Expected board to be numpy array or list but got " + type(board))
        if board.shape != (5,5):
            raise ValueError('Input shape is not 5x5 but ' + str(board.shape))

        self.board = board.astype(int)
        self.shape = self.board.shape
        self.marked = []
        self.unmarked = list(self.board.reshape(5*5))
        self._row_scores = {r: 0 for r in range(self.shape[0])}
        self._col_scores = {c: 0 for c in range(self.shape[1])}
        self.winner = {}
        self.score = None

    def check_number(self, num: Union[int, str]):
        if isinstance(num, str):
            num = int(num)
        results = np.where(self.board == num)
        if len(results[0]) + len(results[1]) == 0:
            return False
        if not num in self.marked:
            self.marked.append(num)
            self.unmarked.remove(num)
            for row in results[0]:
                self._row_scores[row] += 1
            for col in results[1]:
                self._col_scores[col] += 1
            self.check_win()
            return True

    def reset(self):
        self.__init__(self.board)

    def check_win(self):
        if self.winner != {}:
            return True

        for row, score in self._row_scores.items():
            if score == self.shape[1]:
                self.winner['row'] = row
                self._calculate_score()
                return True
        for col, score in self._col_scores.items():
            if score == self.shape[1]:
                self.winner['col'] = col
                self._calculate_score()
                return True
        return False

    def _calculate_score(self):
        if self.check_win():
            self.score = np.sum(self.unmarked)*self.marked[-1]
        

In [269]:
import re

puzzles = []
puzzle_count = 0
for row in bingo[1:]:
    if row == '\n':
        if puzzle_count > 0:
            puzzles.append(BingoBoard(puzzle))
        puzzle = []
        puzzle_count += 1
        continue
    numbers = re.findall('[0-9]*', row)
    numbers = [num.strip() for num in numbers if len(num) > 0]
    puzzle.append(numbers)
        

In [270]:
for draw in marked:
    for board in puzzles:
        prev_score_state = board.score
        board.check_number(draw)
        if board.score != prev_score_state:
            print(board.score)

41503
61664
19395
30285
8646
33984
48320
56640
41028
10296
8805
10890
20943
5128
18780
25890
33524
26792
50765
29640
38740
25090
43940
12789
12369
2988
2550
38610
30960
54540
35010
41040
42660
20184
27540
26520
2044
1648
1780
2028
1728
22572
9652
1467
1983
1338
1374
1059
19694
23134
24553
24596
52638
14652
17512
18050
18450
23000
19147
13448
41184
50112
45504
6860
8100
8760
9340
6480
19344
21638
24924
10478
7733
5092
29393
34671
44135
8855
8947
5037
6187
11339
9972
14328
15732
7848
22654
21712
27512
25004
14353
9548
11842
17286
3432
3850
0
442
3178


# Part Two

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?

## Solution

The solution to `part two` is already given in the first part! 🚀

It's the last score in the list 😉